Underneath, in this notebook, you may find the prepared, e.t. filled with proper math and logic dataset, which performs greatly (>top 3%) even with untuned Deep Learning Model, which for this task is not suitable (obviously it do requires ML model, due to the highly dispersed low number of elements data datasets).

In [18]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab.data_table import DataTable
pd.set_option('display.max_columns', None)
DataTable.max_columns = 30

In [19]:
#Getting the data from github
!wget -O test.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
!wget -O train.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
test_data = pd.read_csv('../content/test.csv')
train_data = pd.read_csv('../content/train.csv')

--2024-08-19 20:57:27--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28210 (28K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  27.55K  --.-KB/s    in 0.002s  

2024-08-19 20:57:27 (15.3 MB/s) - ‘test.csv’ saved [28210/28210]

--2024-08-19 20:57:27--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 

In [20]:
finding_patterns = pd.concat([train_data, test_data], axis=0)
train_data['Age'].fillna(value=finding_patterns['Age'].mean(), inplace=True)
train_data['Fare'].fillna(value=finding_patterns['Fare'].mean(), inplace=True)
test_data['Age'].fillna(value=finding_patterns['Age'].mean(), inplace=True)
test_data['Fare'].fillna(value=finding_patterns['Fare'].mean(), inplace=True)
train_data['Embarked'].fillna(value='X', inplace=True)
train_data['LetterInCabin'] = train_data['Cabin'].str[0]
test_data['LetterInCabin'] = test_data['Cabin'].str[0]
train_data.drop(['Cabin'], axis=1, inplace=True)
test_data.drop(['Cabin'], axis=1, inplace=True)
train_data['Name'].str.extract('([A-Za-z]+)\.', expand=True)

,0
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
...,...
886,Rev
887,Miss
888,Miss
889,Mr


In [21]:
train_data['Name'].head(60).str.extract('([A-Za-z]+)\.', expand=True).value_counts() #Fa female more liekly Fb less likely to survive, --> Ma Mb
df = train_data['Name'].str.len().value_counts()
# for words_num in range(len(num_list)):
# for name in range(len(df)):
#I am trying to find on how to split the line for Fa Fb Ma and Mb
'''
df = df.to_dict()
counter_dict = {key: 0 for key in df.keys()}
sex_divided = train_data.groupby('Sex')['Survived'].value_counts()
for row in range(len(train_data)):
    name_length = len(train_data.loc[row, 'Name'])
    survived = int(train_data.loc[row, 'Survived'])

    # Only increment the counter if the name length matches the key
    if name_length in counter_dict:
        counter_dict[name_length] += survived
# print(df)

#To 11 yo kids, so kids
# I must divide two sets for women and men!!, then I will be able to say the barrier in which names are important in terms of count for the survival (n dividing INTEGER), Mr. Mrs. are measured differently, Encoded on Mrs...,
# Fa, Fb, Ma, Mb,  Normalization for the data will be better! It will be better for Deep Learning Algorithm, 'stead of ML
'''
# total_dict = sum(counter_dict.values())
# sex_divided
kids = train_data[train_data['Age'] < 12]
kids # Important one
train_data['Name'] = train_data['Name'].str.extract('([A-Za-z]+)\.', expand=True)
train_data.loc[train_data['Age'] < 12, 'Name'] = 'Kid'

In [22]:
train_data.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [23]:
train_data['Survived'].value_counts()

,count
Survived,
0,549
1,342


In [24]:
train_data['Name'].head(40)

,Name
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr
5,Mr
6,Mr
7,Kid
8,Mrs
9,Mrs
